500 hPa Vorticity Advection
===========================

Plot an 500-hPa map with calculating vorticity advection using MetPy calculations.

Use Xarray module to read in model data from nomads server. Beyond just plotting 500-hPa level data, this uses calculations from `metpy.calc` to find the vorticity and vorticity advection. 

Adapted from: https://unidata.github.io/python-training/gallery/500hpa_vorticity_advection/ and https://unidata.github.io/python-training/gallery/xarray_500hpa_map/

Imports

In [ ]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import numpy as np
import xarray as xr
import pandas as pd

import metpy.calc as mpcalc
from metpy.units import units

Accessing GFS analysis data from a remote server using Xarray
---------------------------

This is very useful because you don't need to download files locally! Instead, you can access the data on a remote THREDDS server, saving work and disk space.

In [ ]:
# Specify our date/time of product desired
dt = datetime(2021, 12, 11, 6)  ## year, month, day, hour, minute, second

archived = True ### is this an old time (more than ~1 month ago), where we need to get from NCEI?

if archived==True:
    #########for archived data, use NCEI below:
    base_url = "https://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files/"
    data = xr.open_dataset(f'{base_url}{dt:%Y%m}/{dt:%Y%m%d}/'
                           f'gfs_4_{dt:%Y%m%d}_{dt:%H}00_000.grb2',
                           decode_times=True)

else:
    ########### for near-real-time data, use unidata thredds
    base_url = "https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p5deg_ana/"
    data = xr.open_dataset(f'{base_url}GFS_Global_0p5deg_ana_{dt:%Y%m%d}_{dt:%H}00.grib2',
                           decode_times=True)



Get the time in the data file as a pandas datetime

In [ ]:
vtime = pd.to_datetime(data.time)[0]
vtime


Get relevant variables, parse them

In [ ]:
data_subset = data.metpy.parse_cf([
    'u-component_of_wind_isobaric',
    'v-component_of_wind_isobaric',
    'Geopotential_height_isobaric'
]).sel(time=vtime).squeeze()

## subset to North America
data_subset = data_subset.sel(lon=slice(205,310.), lat=slice(68.,18.))    

## define lat and lon variables for convenience
lat = data_subset.lat
lon = data_subset.lon

Inspect data:

In [ ]:
data_subset

Get 500-hPa fields, smooth, and calculate vorticity
----------------------

In [ ]:
### bring in 500-hPa height, smooth it
hght_500 = data_subset['Geopotential_height_isobaric'].metpy.sel(vertical=500*units.hPa)
uwnd_500 = data_subset['u-component_of_wind_isobaric'].metpy.sel(vertical=500*units.hPa)
vwnd_500 = data_subset['v-component_of_wind_isobaric'].metpy.sel(vertical=500*units.hPa)

## smooth the height field with a 5-point smoother
hght_500 = mpcalc.smooth_n_point(hght_500, 5)
### calculate absolute vorticity with metpy and use 9-point smoother on the output (more aggressive smoothing)
avor_500 = mpcalc.smooth_n_point(mpcalc.absolute_vorticity(uwnd_500, vwnd_500),9)

### and calculate the advection and multiply by 10^9
vort_adv = mpcalc.advection(avor_500, u=uwnd_500, v=vwnd_500) * 1e9


And make the map
---------------

In [ ]:
# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=45, central_longitude=-100)

fig = plt.figure(1, figsize=(20,16))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, .02], bottom=.07, top=.99,
                       hspace=0.01, wspace=0.01)
ax = plt.subplot(gs[0], projection=plotcrs)

# Plot Titles
plt.title(r'GFS analysis 500-hPa Heights (m), AVOR$*10^5$ ($s^{-1}$), AVOR Adv$*10^8$ ($s^{-2}$)',
          loc='left', fontsize=18)
plt.title(vtime.strftime("%H%M UTC %d %b %Y"), loc='right', fontsize=18)

# Plot Background
#ax.set_extent([217.5, 290., 18., 66.])
ax.set_extent([227.5, 290., 20., 56.])
ax.coastlines('50m', edgecolor='black', linewidth=0.75)
ax.add_feature(cfeature.STATES, linewidth=.5)

# Plot Height Contours
clev500 = np.arange(5100, 6061, 60)
cs = ax.contour(lon, lat, hght_500, clev500, colors='black', linewidths=1.0,
                linestyles='solid', transform=ccrs.PlateCarree())
plt.clabel(cs, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot Absolute Vorticity Contours, multiply by 10^5 for appropriate scaling
clevvort500 = np.arange(-9, 50, 5)
cs2 = ax.contour(lon, lat, avor_500*1e5, clevvort500, colors='grey',
                 linewidths=1.25, linestyles='dashed', transform=ccrs.PlateCarree())
plt.clabel(cs2, fontsize=10, inline=1, inline_spacing=10, fmt='%i',
           rightside_up=True, use_clabeltext=True)

# Plot Colorfill of Vorticity Advection
clev_avoradv = np.arange(-30, 31, 5)
cf = ax.contourf(lon, lat, vort_adv, clev_avoradv[clev_avoradv != 0], extend='both',
                 cmap='bwr', transform=ccrs.PlateCarree())
#cax = plt.subplot(gs[1])
cb = plt.colorbar(cf, orientation='horizontal', ticks=clev_avoradv,
                 shrink=0.9, pad=0.02, aspect=35)
cb.set_label(r'$1/s^2$', size='large')

# Plot Wind Barbs
# Transform Vectors and plot wind barbs.
ax.barbs(lon, lat, uwnd_500.values, vwnd_500.values, length=6, regrid_shape=20,
         pivot='middle', transform=ccrs.PlateCarree())

### uncomment these if you'd like to save a file
#outfile="500vort_gfs_"+vtime.strftime('%Y%m%d')+"_"+vtime.strftime('%H%M')+".png"
#fig.savefig(outfile, dpi=255, bbox_inches='tight', transparent=False, facecolor='white')

plt.show()   ### show in the notebook

plt.close('all')